In [ ]:
import os
import cv2
import numpy as np
import sounddevice as sd
import librosa
import threading
import tkinter as tk
from tkinter import font as tkfont
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
from scipy.spatial.distance import cosine
from keras_vggface.utils import preprocess_input
import tensorflow as tf
# Load models and embeddings
liveness_model = load_model('liveness_model.keras', compile=False)
face_model = load_model('face_verification_model.keras', compile=False)
face_embeddings = np.load('my_embeddings.npy')
face_labels = np.load('my_labels.npy')

# Global Variables for GUI elements
record_button = None
status_label = None
video_label = None
window = None

# Function to preprocess frame for liveness detection
def preprocess_frame(frame):
    frame_resized = cv2.resize(frame, (224,224))
    frame_normalized = frame_resized / 255.0
    frame_expanded = np.expand_dims(frame_normalized, axis=0)
    return frame_expanded

# Function to predict liveness
def predict_liveness(frame, model):
    preprocessed_frame = preprocess_frame(frame)
    prediction = model.predict(preprocessed_frame)
    return prediction[0][0]

# Function to detect faces in the frame
def detect_faces(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    return faces

# Function to extract and preprocess detected faces
def extract_faces(frame, faces):
    face_images = []
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_images.append(face)
    return face_images

# Function to get embedding from the face
def preprocess_image(image):
    image_resized = cv2.resize(image, (224, 224))
    image_array = np.array(image_resized, dtype=np.float32)
    image_array = preprocess_input(image_array, version=2)
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

def get_embedding(image, model):
    image_preprocessed = preprocess_image(image)
    embedding = model.predict(image_preprocessed)
    return embedding.flatten()

# Function to find closest matching face
def find_closest_face(query_embedding, face_embeddings, face_labels, threshold=0.3):
    min_distance = float("inf")
    closest_label = None
    for i, stored_embedding in enumerate(face_embeddings):
        distance = cosine(query_embedding, stored_embedding)
        if distance < min_distance:
            min_distance = distance
            closest_label = face_labels[i]
    
    if min_distance < threshold:
        return closest_label
    return None

# Function to record audio
def record_test():
    #print("Recording for 3 seconds...")
    update_status("Recording...")
    window.update()  
    audio = sd.rec(int(3 * 16000), samplerate=16000, channels=1)
    sd.wait()
    print("Recording finished.")
    return audio.flatten()

# Function to extract MFCC features from audio
def extract_mfcc(audio, sample_rate=16000):
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, 100 - mfcc.shape[1])), mode='constant')
    return mfcc.flatten()

# Function to load voice embeddings from a batch
def load_batch(batch_num):
    embeddings = np.load(f"embeddings_batch_{batch_num}.npy")
    file_paths = np.load(f"file_paths_batch_{batch_num}.npy")
    return embeddings, file_paths

# Function to find the closest matching voice in a batch
def find_closest_match_in_batch(embedding, embeddings, file_paths):
    min_distance = float("inf")
    closest_file = None
    for i, stored_embedding in enumerate(embeddings):
        distance = cosine(embedding, stored_embedding)
        if distance < min_distance:
            min_distance = distance
            closest_file = file_paths[i]
    return closest_file, min_distance

# Function to find the closest matching voice across all batches
def find_closest_match_across_batches(real_time_embedding, num_batches, threshold=0.5):
    closest_file = None
    min_distance = float("inf")
    for batch_num in range(num_batches):
        try:
            embeddings, file_paths = load_batch(batch_num)
        except FileNotFoundError:
            print(f"Batch file embeddings_batch_{batch_num}.npy not found. Skipping.")
            continue
        
        batch_closest_file, batch_min_distance = find_closest_match_in_batch(real_time_embedding, embeddings, file_paths)
        if batch_min_distance < min_distance:
            min_distance = batch_min_distance
            closest_file = batch_closest_file
    
    if min_distance < threshold:
        return closest_file
    return None

# Function to trigger voice recording and verification
def start_voice_verification():
    record_button.pack_forget()  # Hide the button while recording
    threading.Thread(target=voice_verification_thread).start()

def voice_verification_thread():
    real_time_audio = record_test()
    real_time_embedding = extract_mfcc(real_time_audio)
    closest_file = find_closest_match_across_batches(real_time_embedding, num_batches=43, threshold=0.5)

    if closest_file:
        update_status(f"Access Granted: Voice recognized from {closest_file}")
        refresh_display_after_unlock()
    else:
        update_status("Access Denied: No Voice Match Found")


def real_time_detection():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        update_status("Error: Could not open video capture.")
        return

    processing = True
    while processing:
        ret, frame = cap.read()
        if not ret or frame is None:
            update_status("Error: Failed to capture image.")
            break
        
        try:
            liveness_score = predict_liveness(frame, liveness_model)
            liveness_threshold = 0.5
            if liveness_score > liveness_threshold:
                faces = detect_faces(frame)
                if len(faces) > 0:
                    face_images = extract_faces(frame, faces)
                    for face in face_images:
                        face_embedding = get_embedding(face, face_model)
                        matching_label = find_closest_face(face_embedding, face_embeddings, face_labels, threshold=0.3)
                        
                        if matching_label:
                            update_status(f"Access Granted: {matching_label} recognized")
                            processing = False
                            refresh_display_after_unlock()
                        else:
                            update_status("Face Not Recognized: Proceeding to Voice Verification")
                            record_button.pack(pady=20)  # Show the button when needed
                            processing = False  # Stop processing to wait for voice verification
                else:
                    update_status("No face detected.")
                    processing = False
            else:
                update_status("Access Denied: Fake Image Detected")
                processing = False
            
            # Convert frame for display
            cv2_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(cv2_frame)
            img = ImageTk.PhotoImage(img)
            video_label.config(image=img)
            video_label.image = img

        except cv2.error as e:
            update_status(f"Error processing frame: {e}")
            break
        
        window.update()  # Ensure GUI updates while processing

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
# Function to trigger voice recording and verification
def start_voice_verification():
    update_status("Recording voice...")
    window.update()
    real_time_audio = record_test()
    real_time_embedding = extract_mfcc(real_time_audio)
    closest_file = find_closest_match_across_batches(real_time_embedding, num_batches=43, threshold=0.5)

    if closest_file:
        update_status(f"Access Granted: Voice recognized from {closest_file}")
    else:
        update_status("Access Denied: No Voice Match Found")
# GUI-related functions
def update_status(status_text):
    window.after(0, lambda: status_label.config(text=status_text))

def refresh_display_after_unlock():
    # Wait for 5 seconds and then clear the displayed image
    window.after(10000, clear_display)

def clear_display():
    video_label.config(image='')  # Clear the image
    update_status("")  # Clear the status

def start_detection_thread():
    global detection_thread
    detection_thread = threading.Thread(target=real_time_detection, daemon=True)
    detection_thread.start()

def exit_program():
    global running
    running = False
    if detection_thread.is_alive():
        detection_thread.join()
    window.destroy()

def create_gui():
    global window, status_label, video_label,record_button

    window = tk.Tk()
    window.title("Office Entry System")
    window.configure(bg='lightblue')

    canvas = tk.Canvas(window, width=800, height=600, bg='lightblue')
    canvas.pack(side="left", fill="both", expand=True)

    scrollbar = tk.Scrollbar(window, orient="vertical", command=canvas.yview)
    scrollbar.pack(side="right", fill="y")
    canvas.configure(yscrollcommand=scrollbar.set)

    scrollable_frame = tk.Frame(canvas, bg='lightblue')
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    custom_font = tkfont.Font(family="Helvetica", size=16, weight="bold")

    label = tk.Label(scrollable_frame, text="Office Entry System", font=custom_font, bg="lightblue", fg="darkblue")
    label.pack(pady=20)

    video_label = tk.Label(scrollable_frame)
    video_label.pack(pady=10)

    start_button = tk.Button(scrollable_frame, text="Start Security System", font=custom_font, command=start_detection_thread)
    start_button.pack(pady=20)

    record_button = tk.Button(scrollable_frame, text="Record Voice", font=custom_font, command=start_voice_verification)
    record_button.pack_forget() 

    exit_button = tk.Button(scrollable_frame, text="Exit", font=custom_font, command=exit_program)
    exit_button.pack(pady=10)

    status_label = tk.Label(scrollable_frame, text="", font=custom_font, bg='lightblue', fg="darkred")
    status_label.pack(pady=10)

    window.protocol("WM_DELETE_WINDOW", exit_program)

    window.mainloop()

# Start the GUI application
create_gui()


1/1 [==============================] - 0s 126ms/step
Recording finished.
Batch file embeddings_batch_13.npy not found. Skipping.


In [11]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from scipy.spatial.distance import cosine
from keras_vggface.utils import preprocess_input

# Load models and embeddings
liveness_model = load_model('liveness_model.keras', compile=False)
face_model = load_model('face_verification_model.keras', compile=False)
face_embeddings = np.load('my_embeddings.npy')  # Precomputed face embeddings
face_labels = np.load('my_labels.npy')  # Labels for embeddings

# Function to preprocess the face for the model
def preprocess_frame(face_image):
    face_resized = cv2.resize(face_image, (224, 224))  # Resize to the input size of the liveness model
    face_normalized = face_resized / 255.0  # Normalize the pixel values
    face_expanded = np.expand_dims(face_normalized, axis=0)  # Add batch dimension
    return face_expanded

# Liveness detection for a single face image
def predict_liveness(face_image, model):
    preprocessed_face = preprocess_frame(face_image)
    prediction = model.predict(preprocessed_face)
    return prediction[0][0]  # Return the liveness score

# Function to preprocess the face for face verification
def preprocess_image_for_verification(face_image):
    face_resized = cv2.resize(face_image, (224, 224))  # Resize to the input size of the face model
    face_array = np.array(face_resized, dtype=np.float32)
    face_array = preprocess_input(face_array, version=2)
    face_array = np.expand_dims(face_array, axis=0)  # Add batch dimension
    return face_array

# Function to extract embedding from the face
def get_embedding(face_image, model):
    preprocessed_face = preprocess_image_for_verification(face_image)
    embedding = model.predict(preprocessed_face)
    return embedding.flatten()

# Function to find the closest matching face based on embeddings
def find_closest_face(query_embedding, face_embeddings, face_labels, threshold=0.3):
    min_distance = float("inf")
    closest_label = None
    for i, stored_embedding in enumerate(face_embeddings):
        distance = cosine(query_embedding, stored_embedding)
        if distance < min_distance:
            min_distance = distance
            closest_label = face_labels[i]
    
    if min_distance < threshold:
        return closest_label
    return None

# Real-time detection and verification process
def real_time_detection(liveness_model, face_model, face_embeddings, face_labels):
    cap = cv2.VideoCapture(0)  # Open default camera

    if not cap.isOpened():
        print("Error: Could not open video capture.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Step 1: Detect faces in the frame
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        if len(faces) > 0:
            for (x, y, w, h) in faces:
                face = frame[y:y+h, x:x+w]

                # Step 2: Ignore small faces
                if w < 100 or h < 100:
                    print(f"Ignoring small face of size: {w}x{h}")
                    continue

                # Step 3: Liveness Detection
                liveness_score = predict_liveness(face, liveness_model)
                print(f"Liveness score: {liveness_score}")
                if liveness_score > 0.5:
                    label = 'Real'
                    color = (0, 255, 0)  # Green for real
                else:
                    label = 'Fake'
                    color = (0, 0, 255)  # Red for fake
                    cv2.putText(frame, 'Fake Face Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                    continue  # Skip to the next face if fake

                # Step 4: Face Verification (only for real faces)
                face_embedding = get_embedding(face, face_model)
                matching_label = find_closest_face(face_embedding, face_embeddings, face_labels, threshold=0.3)
                if matching_label:
                    cv2.putText(frame, f'Access Granted: {matching_label}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                else:
                    cv2.putText(frame, 'Access Denied', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

                # Draw bounding box around face
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

        else:
            print("No face detected.")
        
        # Step 5: Show the frame with bounding boxes and labels
        cv2.imshow('Liveness Detection and Face Verification', frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run real-time detection with liveness and face verification
real_time_detection(liveness_model, face_model, face_embeddings, face_labels)


1/1 [==============================] - 2s 2s/step
Liveness score: 0.9999964833259583
1/1 [==============================] - 0s 112ms/step
Liveness score: 0.9999985098838806
1/1 [==============================] - 0s 105ms/step
Liveness score: 0.9999992847442627
1/1 [==============================] - 0s 101ms/step
Liveness score: 0.9999933242797852
1/1 [==============================] - 0s 97ms/step
Liveness score: 0.9996287226676941
1/1 [==============================] - 0s 121ms/step
Liveness score: 0.9999896883964539
1/1 [==============================] - 0s 95ms/step
Liveness score: 0.9999599456787109
1/1 [==============================] - 0s 204ms/step


In [ ]:
import cv2
import numpy as np
import threading
import sounddevice as sd
import librosa
import tkinter as tk
from tkinter import font as tkfont
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
from scipy.spatial.distance import cosine
from keras_vggface.utils import preprocess_input

# Load models and embeddings
liveness_model = load_model('liveness_model.keras', compile=False)
face_model = load_model('face_verification_model.keras', compile=False)
face_embeddings = np.load('my_embeddings.npy')  # Precomputed face embeddings
face_labels = np.load('my_labels.npy')  # Labels for embeddings

# Global variables for GUI
window = None
video_label = None
status_label = None
record_button = None

# Preprocess face image for liveness detection
def preprocess_frame(face_image):
    face_resized = cv2.resize(face_image, (224, 224))
    face_normalized = face_resized / 255.0
    face_expanded = np.expand_dims(face_normalized, axis=0)
    return face_expanded

# Liveness detection function
def predict_liveness(face_image, model):
    preprocessed_face = preprocess_frame(face_image)
    prediction = model.predict(preprocessed_face)
    return prediction[0][0]

# Preprocess face image for verification
def preprocess_image_for_verification(face_image):
    face_resized = cv2.resize(face_image, (224, 224))
    face_array = np.array(face_resized, dtype=np.float32)
    face_array = preprocess_input(face_array, version=2)
    face_array = np.expand_dims(face_array, axis=0)
    return face_array

# Get face embedding
def get_embedding(face_image, model):
    preprocessed_face = preprocess_image_for_verification(face_image)
    embedding = model.predict(preprocessed_face)
    return embedding.flatten()

# Find closest face match
def find_closest_face(query_embedding, face_embeddings, face_labels, threshold=0.3):
    min_distance = float("inf")
    closest_label = None
    for i, stored_embedding in enumerate(face_embeddings):
        distance = cosine(query_embedding, stored_embedding)
        if distance < min_distance:
            min_distance = distance
            closest_label = face_labels[i]
    
    if min_distance < threshold:
        return closest_label
    return None

# Voice recording and verification
def record_test():
    update_status("Recording for 3 seconds...")
    window.update()
    audio = sd.rec(int(3 * 16000), samplerate=16000, channels=1)
    sd.wait()  # Wait until recording is finished
    return audio.flatten()

# Extract MFCC features from audio
def extract_mfcc(audio, sample_rate=16000):
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, 100 - mfcc.shape[1])), mode='constant')
    return mfcc.flatten()

# Load voice embeddings
def load_batch(batch_num):
    embeddings = np.load(f"embeddings_batch_{batch_num}.npy")
    file_paths = np.load(f"file_paths_batch_{batch_num}.npy")
    return embeddings, file_paths

# Find the closest voice match
def find_closest_match_in_batch(embedding, embeddings, file_paths):
    min_distance = float("inf")
    closest_file = None
    for i, stored_embedding in enumerate(embeddings):
        distance = cosine(embedding, stored_embedding)
        if distance < min_distance:
            min_distance = distance
            closest_file = file_paths[i]
    return closest_file, min_distance

# Function to trigger voice verification
def start_voice_verification():
    update_status("Recording voice...")
    window.update()
    real_time_audio = record_test()
    real_time_embedding = extract_mfcc(real_time_audio)
    closest_file = find_closest_match_across_batches(real_time_embedding, num_batches=43, threshold=0.5)

    if closest_file:
        update_status(f"Access Granted: Voice recognized from {closest_file}")
    else:
        update_status("Access Denied: No Voice Match Found")

# Voice verification across batches
def find_closest_match_across_batches(real_time_embedding, num_batches, threshold=0.5):
    closest_file = None
    min_distance = float("inf")
    for batch_num in range(num_batches):
        embeddings, file_paths = load_batch(batch_num)
        batch_closest_file, batch_min_distance = find_closest_match_in_batch(real_time_embedding, embeddings, file_paths)
        if batch_min_distance < min_distance:
            min_distance = batch_min_distance
            closest_file = batch_closest_file
    
    if min_distance < threshold:
        return closest_file
    return None

# Real-time face detection, liveness, and verification
def real_time_detection():
    cap = cv2.VideoCapture(0)  # Open the default camera

    if not cap.isOpened():
        update_status("Error: Could not open video capture.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            update_status("Error: Failed to capture image.")
            break

        # Step 1: Detect faces
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        if len(faces) > 0:
            for (x, y, w, h) in faces:
                face = frame[y:y+h, x:x+w]

                # Step 2: Ignore small faces
                if w < 100 or h < 100:
                    update_status(f"Ignoring small face of size: {w}x{h}")
                    continue

                # Step 3: Liveness detection
                liveness_score = predict_liveness(face, liveness_model)
                if liveness_score > 0.5:
                    label = 'Real'
                    color = (0, 255, 0)  # Green for real
                else:
                    label = 'Fake'
                    color = (0, 0, 255)  # Red for fake
                    update_status('Fake Face Detected')
                    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                    continue

                # Step 4: Face verification
                face_embedding = get_embedding(face, face_model)
                matching_label = find_closest_face(face_embedding, face_embeddings, face_labels, threshold=0.3)
                if matching_label:
                    update_status(f'Access Granted: {matching_label}')
                else:
                    update_status('Face Not Recognized: Proceeding to Voice Verification')
                    record_button.pack(pady=20)
                    return

                # Draw bounding box around face
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

        else:
            update_status("No face detected.")
        
        show_frame_in_gui(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# GUI update functions
def update_status(status_text):
    window.after(0, lambda: status_label.config(text=status_text))

def show_frame_in_gui(frame):
    cv2_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2_frame)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

# Start real-time detection in a separate thread
def start_detection_thread():
    detection_thread = threading.Thread(target=real_time_detection, daemon=True)
    detection_thread.start()

# GUI creation
def create_gui():
    global window, video_label, status_label, record_button

    window = tk.Tk()
    window.title("Office Entry System")
    window.configure(bg='lightblue')

    canvas = tk.Canvas(window, width=800, height=600, bg='lightblue')
    canvas.pack(side="left", fill="both", expand=True)

    scrollbar = tk.Scrollbar(window, orient="vertical", command=canvas.yview)
    scrollbar.pack(side="right", fill="y")
    canvas.configure(yscrollcommand=scrollbar.set)

    scrollable_frame = tk.Frame(canvas, bg='lightblue')
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    custom_font = tkfont.Font(family="Helvetica", size=16, weight="bold")

    label = tk.Label(scrollable_frame, text="Office Entry System", font=custom_font, bg="lightblue", fg="darkblue")
    label.pack(pady=20)

    video_label = tk.Label(scrollable_frame)
    video_label.pack(pady=10)

    start_button = tk.Button(scrollable_frame, text="Start Security System", font=custom_font, command=start_detection_thread)
    start_button.pack(pady=20)

    record_button = tk.Button(scrollable_frame, text="Record Voice", font=custom_font, command=start_voice_verification)
    record_button.pack_forget()  # Initially
    

    exit_button = tk.Button(scrollable_frame, text="Exit", font=custom_font, command=exit_program)
    exit_button.pack(pady=10)

    status_label = tk.Label(scrollable_frame, text="", font=custom_font, bg='lightblue', fg="darkred")
    status_label.pack(pady=10)

    window.protocol("WM_DELETE_WINDOW", exit_program)

    window.mainloop()

# Start the GUI application
create_gui()



1/1 [==============================] - 0s 214ms/step
